In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import time
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Dropout
from keras import Sequential
from sklearn.metrics import classification_report
from keras.layers import Conv2D,Flatten,MaxPool2D,BatchNormalization,Activation,Dropout
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image
import csv
import mediapipe as mp

In [2]:
df = pd.read_csv("landmark2.csv",on_bad_lines='skip')
X = df.drop(df.columns[0], axis = 1)
y = df[df.columns[0]]

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

len(y_train.unique())

#y_train=keras.utils.to_categorical(y_train,27)
#y_test=keras.utils.to_categorical(y_test)

27

In [4]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(units=63, input_shape=[63]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=27, activation='softmax')
])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 63)                4032      
                                                                 
 dropout (Dropout)           (None, 63)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8192      
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 512)               131584    
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0

In [5]:
y_train_encoded = pd.get_dummies(y_train, prefix='target')
y_test_encoded = pd.get_dummies(y_test, prefix='target')
y_train_encoded.head(3)

,target_A,target_B,target_C,target_D,target_E,target_F,target_G,target_H,target_I,target_K,...,target_S,target_T,target_U,target_V,target_W,target_X,target_Y,target_d,target_n,target_s
39877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5907,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41920,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

In [7]:
model1.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [8]:
start = time.time()

history = model1.fit(
    X_train,
    y_train_encoded,
    validation_split =0.2,
    epochs=5,
    callbacks=[early_stop]
)

stop1 = time.time()


y_pred = model1.predict(X_test)

stop2 = time.time()
apprentissage_RFC = round(stop1-start,3)
prediction_RFC = round(stop2-start,3)
print(f"Temps d'apprentissage : {apprentissage_RFC} s")
print(f"Temps d'apprentissage+prediction : {prediction_RFC} s")

Epoch 1/5
1155/1155 [==============================] - 12s 10ms/step - loss: 0.8893 - accuracy: 0.6951 - val_loss: 0.2218 - val_accuracy: 0.9333
Epoch 2/5
1155/1155 [==============================] - 10s 9ms/step - loss: 0.3125 - accuracy: 0.8980 - val_loss: 0.1535 - val_accuracy: 0.9485
Epoch 3/5
1155/1155 [==============================] - 10s 9ms/step - loss: 0.2640 - accuracy: 0.9168 - val_loss: 0.2787 - val_accuracy: 0.9047
Epoch 4/5
1155/1155 [==============================] - 11s 10ms/step - loss: 0.2214 - accuracy: 0.9316 - val_loss: 0.2314 - val_accuracy: 0.9251
Epoch 5/5
482/482 [==============================] - 2s 3ms/step
Temps d'apprentissage : 53.314 s
Temps d'apprentissage+prediction : 55.52 s


In [9]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [10]:
liste_lettre=['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W',"X","Y"]
liste_nb = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

# Create a dictionary that maps letters to numbers
key = {nb: lettre for nb, lettre in zip(liste_nb, liste_lettre)}

# Print the dictionary
print(key)

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'K', 10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P', 15: 'Q', 16: 'R', 17: 'S', 18: 'T', 19: 'U', 20: 'V', 21: 'W', 22: 'X', 23: 'Y'}


In [ ]:
# For webcam input:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
        continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        for hand_no, hand_landmarks in enumerate(results.multi_hand_landmarks):
            coord = []
            for i in range(21):
                
                x = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].x
                y = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].y
                z = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].z
                
                coord.append([x, y, z])
           
            coord_rs = tf.reshape(coord, [1, -1])
            live_pred = model1.predict(coord_rs)
            live_pred = np.argmax(live_pred, axis=1)
            display = key.get(int(live_pred))

            
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame,str(display),(50, 50),font,1,(0, 255, 255))
#             cv2.putText(frame, text, (x, y), font, scale, color)
            
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

    # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
                break

cap.release()